In [ ]:
!pip install gensim
!pip install nltk
!pip install tweet-preprocessor

import nltk
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
# !gzip -d '/content/drive/My Drive/GoogleNews-vectors-negative300.bin.gz'

In [ ]:
corpus_path = '/content/drive/My Drive/GoogleNews-vectors-negative300.bin'


In [ ]:
from nltk.tokenize import sent_tokenize, word_tokenize 
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))
import warnings 
from scipy import spatial
  
warnings.filterwarnings(action = 'ignore') 

from gensim.models import Word2Vec
import gensim
import pandas as pd
import numpy as np

import preprocessor as p




In [ ]:
train_path = '/content/drive/My Drive/AAAI CONSTRAINT/Fake News/Constraint_English_Train - Sheet1.csv'
val_path = '/content/drive/My Drive/AAAI CONSTRAINT/Fake News/Constraint_English_Val - Sheet1.csv'

In [ ]:
train_df = pd.read_csv(train_path)
train_df.head()

,id,tweet,label
0,1,The CDC currently reports 99031 deaths. In gen...,real
1,2,States reported 1121 deaths a small rise from ...,real
2,3,Politically Correct Woman (Almost) Uses Pandem...,fake
3,4,#IndiaFightsCorona: We have 1524 #COVID testin...,real
4,5,Populous states can generate large case counts...,real


In [ ]:
val_df = pd.read_csv(val_path)
val_df.head()

,id,tweet,label
0,1,Chinese converting to Islam after realising th...,fake
1,2,11 out of 13 people (from the Diamond Princess...,fake
2,3,"COVID-19 Is Caused By A Bacterium, Not Virus A...",fake
3,4,Mike Pence in RNC speech praises Donald Trump’...,fake
4,5,6/10 Sky's @EdConwaySky explains the latest #C...,real


In [ ]:
train_df.isnull().sum()

id       0
tweet    0
label    0
dtype: int64

In [ ]:
def subst(s):
    s = s.replace('theres','there is')
    s = s.replace('didnt','did not')
    s = s.replace("didn't",'did not')
    s = s.replace('havent','have not')
    s = s.replace("aren't",'are not')
    s = s.replace("haven't",'have not')
    s = s.replace('hasnt','has not')
    s = s.replace("hasn't",'has not')
    s = s.replace('couldnt','could not')
    s = s.replace("couldn't",'could not')
    s = s.replace("We'd ",'We would ')
    s = s.replace("we'd ",'we would ')
    s = s.replace("I'd",'I would')
    s = s.replace('cant','can not')
    s = s.replace("can't", 'can not')
    s = s.replace('cannot','can not')
    s = s.replace('Im ','I am ')
    s = s.replace("I've ",'I have ')
    s = s.replace("i've ",'I have ')
    s = s.replace("I'm ",'I am ')
    s = s.replace("isnt",'Is not')
    s = s.replace("isn't",'Is not')    
    s = s.replace('wont','will not')
    s = s.replace("won't",'will not')
    s = s.replace('dont','do not')
    s = s.replace("don't",'do not')
    s = s.replace('doesnt','does not')
    s = s.replace("doesn't",'does not')
    return s

In [ ]:
tweets = train_df.tweet.values
val_tweets = val_df.tweet.values
print(tweets)
for i in range(len(tweets)):
  if tweets[i]:
    tweets[i] = subst(tweets[i])
for i in range(len(val_tweets)):
  val_tweets[i] = subst(val_tweets[i])
print(tweets)

['The CDC currently reports 99031 deaths. In general the discrepancies in death counts between different sources are small and explicable. The death toll stands at roughly 100000 people today.'
 'States reported 1121 deaths a small rise from last Tuesday. Southern states reported 640 of those deaths. https://t.co/YASGRTT4ux'
 'Politically Correct Woman (Almost) Uses Pandemic as Excuse Not to Reuse Plastic Bag https://t.co/thF8GuNFPe #coronavirus #nashville'
 ...
 '_A post claims a COVID-19 vaccine has already been developed and will cause widespread infertility._'
 'Aamir Khan Donate 250 Cr. In PM Relief Cares Fund'
 'It has been 93 days since the last case of COVID-19 was acquired locally from an unknown source. The total number of active cases of COVID-19 in New Zealand’s managed isolation and quarantine facilities is now 25.']
['The CDC currently reports 99031 deaths. In general the discrepancies in death counts between different sources are small and explicable. The death toll stan

In [ ]:
clean_tweets = [p.clean(tweet) for tweet in tweets ]
val_clean_tweets = [p.clean(tweet) for tweet in val_tweets ]
print(clean_tweets)

['The CDC currently reports deaths. In general the discrepancies in death counts between different sources are small and explicable. The death toll stands at roughly people today.', 'States reported deaths a small rise from last Tuesday. Southern states reported of those deaths.', 'Politically Correct Woman (Almost) Uses Pandemic as Excuse Not to Reuse Plastic Bag', ': We have testing laboratories in India and as on th August tests have been done : DG', 'Populous states can generate large case counts but if you look at the new cases per million today smaller states are showing more cases per million than California or Texas: AL AR ID KS KY LA MS NV and SC.', 'Covid Act Now found "on average each person in Illinois with COVID-19 is infecting other people. Data shows that the infection growth rate has declined over time this factors in the stay-at-home order and other restrictions put in place."', 'If you tested positive for and have no symptoms stay home and away from other people. Lear

In [ ]:
tokens = [word_tokenize(tweet) for tweet in clean_tweets]
val_tokens = [word_tokenize(tweet) for tweet in val_clean_tweets]
print(tokens)

[['The', 'CDC', 'currently', 'reports', 'deaths', '.', 'In', 'general', 'the', 'discrepancies', 'in', 'death', 'counts', 'between', 'different', 'sources', 'are', 'small', 'and', 'explicable', '.', 'The', 'death', 'toll', 'stands', 'at', 'roughly', 'people', 'today', '.'], ['States', 'reported', 'deaths', 'a', 'small', 'rise', 'from', 'last', 'Tuesday', '.', 'Southern', 'states', 'reported', 'of', 'those', 'deaths', '.'], ['Politically', 'Correct', 'Woman', '(', 'Almost', ')', 'Uses', 'Pandemic', 'as', 'Excuse', 'Not', 'to', 'Reuse', 'Plastic', 'Bag'], [':', 'We', 'have', 'testing', 'laboratories', 'in', 'India', 'and', 'as', 'on', 'th', 'August', 'tests', 'have', 'been', 'done', ':', 'DG'], ['Populous', 'states', 'can', 'generate', 'large', 'case', 'counts', 'but', 'if', 'you', 'look', 'at', 'the', 'new', 'cases', 'per', 'million', 'today', 'smaller', 'states', 'are', 'showing', 'more', 'cases', 'per', 'million', 'than', 'California', 'or', 'Texas', ':', 'AL', 'AR', 'ID', 'KS', 'KY', 

In [ ]:
nonAlnum = []
filterWords = ['.',',','(',')',':','``',"''",'-',';','[',']','?',"'",'!','_',"'s",'','$','%','`','=','<','|']
for i in range(len(tokens)):
  tokens[i] = [word for word in tokens[i] if not word.lower() in stop_words]
  # Split words with -
  newToken = []
  for word in tokens[i]:
    
    if word in filterWords:
      continue
    
    word = word.replace('&','and')
    word = word.replace("'re","are")
    word = word.replace("n't","not")
    word = word.replace("'ve","have")
    word = word.replace("'d","would")
    word = word.replace("'ll'","will")
    word = word.replace('.','')
    word = word.replace('_','')
    word = word.replace("'",'')
    word = word.replace(":",'')
    word = word.replace("~",'')
    word = word.replace("=",'')

    if word in filterWords:
      continue
    
    if word.isalnum():
      newToken.append(word)
      continue
    if '*' in word:
      s = word.split('*')
      s = [x for x in s if x]
      newToken.extend(s)
      continue
    if word.replace(',','').isnumeric():
      newToken.append(word.replace(',',''))
      continue

    if ',' in word:
      s = word.split(',')
      s = [x for x in s if x]
      newToken.extend(s)
      continue
    if '-' in word or '/' in word or '+' in word:
      s = word
      s = " ".join(s.split('/'))
      s = " ".join(s.split('-'))
      s = " ".join(s.split('+'))
      words = s.split(' ')
      words = [x for x in words if x]
      newToken.extend(words)
      continue
    

    newToken.append(word)
  tokens[i] = newToken   

for i in range(len(val_tokens)):
  val_tokens[i] = [word for word in val_tokens[i] if not word.lower() in stop_words]
  # Split words with -
  newToken = []
  for word in val_tokens[i]:
    
    if word in filterWords:
      continue
    
    word = word.replace('&','and')
    word = word.replace("'re","are")
    word = word.replace("n't","not")
    word = word.replace("'ve","have")
    word = word.replace("'d","would")
    word = word.replace("'ll'","will")
    word = word.replace('.','')
    word = word.replace('_','')
    word = word.replace("'",'')
    word = word.replace(":",'')
    word = word.replace("~",'')
    word = word.replace("=",'')

    if word in filterWords:
      continue
    
    if word.isalnum():
      newToken.append(word)
      continue
    if '*' in word:
      s = word.split('*')
      s = [x for x in s if x]
      newToken.extend(s)
      continue
    if word.replace(',','').isnumeric():
      newToken.append(word.replace(',',''))
      continue

    if ',' in word:
      s = word.split(',')
      s = [x for x in s if x]
      newToken.extend(s)
      continue
    if '-' in word or '/' in word or '+' in word:
      s = word
      s = " ".join(s.split('/'))
      s = " ".join(s.split('-'))
      s = " ".join(s.split('+'))
      words = s.split(' ')
      words = [x for x in words if x]
      newToken.extend(words)
      continue
    

    newToken.append(word)
  val_tokens[i] = newToken 

for i in range(len(tokens)):
  nonAlnum.extend([word for word in tokens[i] if not word.isalnum()])
  
print(tokens)
print(val_tokens)
print(len(nonAlnum),nonAlnum)
star = [x for x in nonAlnum if "'" in x]
print(star)
print(len(tokens),len(train_df))


[['CDC', 'currently', 'reports', 'deaths', 'general', 'discrepancies', 'death', 'counts', 'different', 'sources', 'small', 'explicable', 'death', 'toll', 'stands', 'roughly', 'people', 'today'], ['States', 'reported', 'deaths', 'small', 'rise', 'last', 'Tuesday', 'Southern', 'states', 'reported', 'deaths'], ['Politically', 'Correct', 'Woman', 'Almost', 'Uses', 'Pandemic', 'Excuse', 'Reuse', 'Plastic', 'Bag'], ['testing', 'laboratories', 'India', 'th', 'August', 'tests', 'done', 'DG'], ['Populous', 'states', 'generate', 'large', 'case', 'counts', 'look', 'new', 'cases', 'per', 'million', 'today', 'smaller', 'states', 'showing', 'cases', 'per', 'million', 'California', 'Texas', 'AL', 'AR', 'ID', 'KS', 'KY', 'LA', 'MS', 'NV', 'SC'], ['Covid', 'Act', 'found', 'average', 'person', 'Illinois', 'COVID', '19', 'infecting', 'people', 'Data', 'shows', 'infection', 'growth', 'rate', 'declined', 'time', 'factors', 'stay', 'at', 'home', 'order', 'restrictions', 'put', 'place'], ['tested', 'positive

In [ ]:
model = gensim.models.KeyedVectors.load_word2vec_format(corpus_path, binary = True) 

In [ ]:
vector = model['king'] - model['man'] + model['woman']
num_features = len(vector)
print(num_features)
model.similar_by_vector(vector, topn=10, restrict_vocab=None)

300


[('king', 0.8449392318725586),
 ('queen', 0.7300517559051514),
 ('monarch', 0.6454660892486572),
 ('princess', 0.6156251430511475),
 ('crown_prince', 0.5818676948547363),
 ('prince', 0.5777117609977722),
 ('kings', 0.5613663792610168),
 ('sultan', 0.5376776456832886),
 ('Queen_Consort', 0.5344247817993164),
 ('queens', 0.5289887189865112)]

In [ ]:
index2word_set = set(model.wv.index2word)

def avg_feature_vector(sentence, model, num_features, index2word_set):
    
    feature_vec = np.zeros((num_features, ), dtype='float32')
    n_words = 0
    for word in words:
        if word in index2word_set:
            n_words += 1
            feature_vec = np.add(feature_vec, model[word])
    if (n_words > 0):
        feature_vec = np.divide(feature_vec, n_words)
    return feature_vec

In [ ]:
s1_afv = avg_feature_vector('this is a sentence', model=model, num_features=300, index2word_set=index2word_set)
s2_afv = avg_feature_vector('this is also sentence', model=model, num_features=300, index2word_set=index2word_set)
sim = 1 - spatial.distance.cosine(s1_afv, s2_afv)
print(sim)

0.9596675038337708


In [ ]:
tweet_tokens = tokens
for i in range(len(tweet_tokens)):
  tweet_tokens[i] = [x for x in tweet_tokens[i] if x in model.vocab]
  
val_tweet_tokens = val_tokens
for i in range(len(val_tweet_tokens)):
  val_tweet_tokens[i] = [x for x in val_tweet_tokens[i] if x in model.vocab]


train_X = []
val_X = []
for i in range(len(tweet_tokens)):
  vector = avg_feature_vector(" ".join(tweet_tokens[i]),model=model, num_features=300, index2word_set=index2word_set)
  train_X.append(vector)
for i in range(len(val_tweet_tokens)):
  vector = avg_feature_vector(" ".join(val_tweet_tokens[i]),model=model, num_features=300, index2word_set=index2word_set)
  val_X.append(vector)
print(len(train_X))
print(len(val_X))



6420
2140


In [ ]:
labels = train_df.label.values
val_labels = val_df.label.values
print(labels[:20])
train_Y = []
val_Y = []
for x in labels:
  if x == 'real':
    train_Y.append(1)
  else:
    train_Y.append(0)
for x in val_labels:
  if x == 'real':
    val_Y.append(1)
  else:
    val_Y.append(0)


print(len(train_Y))
print(train_Y.count(1),train_Y.count(0))
print(len(val_Y))
print(val_Y.count(1),val_Y.count(0))
print(train_Y)

['real' 'real' 'fake' 'real' 'real' 'real' 'real' 'fake' 'fake' 'fake'
 'real' 'fake' 'real' 'real' 'real' 'real' 'fake' 'real' 'real' 'fake']
6420
3360 3060
2140
1120 1020
[1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1

In [ ]:
train_X = np.array(train_X)
train_Y = np.array(train_Y)

assert len(train_X) == 6420
assert len(train_Y) == 6420
assert len(train_X[0]) == 300


In [ ]:
from sklearn.linear_model import LogisticRegression

reg = LogisticRegression().fit(train_X, train_Y)

In [ ]:
print("Train Accuracy = ", reg.score(train_X, train_Y))
print("Val Accuracy = ", reg.score(val_X, val_Y))

Train Accuracy =  0.9096573208722741
Val Accuracy =  0.905607476635514


In [ ]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC

clf = make_pipeline(StandardScaler(), SVC(gamma='auto'))
clf.fit(train_X, train_Y)

Pipeline(memory=None,
         steps=[('standardscaler',
                 StandardScaler(copy=True, with_mean=True, with_std=True)),
                ('svc',
                 SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None,
                     coef0=0.0, decision_function_shape='ovr', degree=3,
                     gamma='auto', kernel='rbf', max_iter=-1, probability=False,
                     random_state=None, shrinking=True, tol=0.001,
                     verbose=False))],
         verbose=False)

In [ ]:
print("Train Accuracy = ",clf.score(train_X,train_Y))
print("Val Accuracy = ", clf.score(val_X,val_Y))


Train Accuracy =  0.9716510903426792
Val Accuracy =  0.9271028037383178


In [ ]:
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(max_depth=8, random_state=0, n_estimators=100)
clf.fit(train_X, train_Y)



NameError: ignored

In [ ]:
print("Train Accuracy = ",clf.score(train_X,train_Y))
print("Val Accuracy = ", clf.score(val_X,val_Y))


In [ ]:
from numpy import save

save('train_X.npy', train_X)
save('train_Y.npy', train_Y)
save('val_X.npy', val_X)
save('val_Y.npy', val_Y)